<a href="https://colab.research.google.com/github/cengc13/2040FinalProject/blob/master/src/models/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model<a id="1"></a>

In [0]:
import matplotlib.pyplot as plt
plt.style.use('seaborn')
plt.rcParams['axes.grid'] = False
%matplotlib inline

In [21]:
import os

import numpy as np
import pandas as pd
try:
    from google.colab import drive
    drive.mount('/content/drive')
    %tensorflow_version 2.x
    IS_COLAB = True
except:
    IS_COLAB = False
    pass
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import (ModelCheckpoint, ReduceLROnPlateau, 
                                        CSVLogger, Callback)
import tensorflow.keras.backend as K
assert tf.__version__ >= '2.0'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
! pip install -U tokenizers==0.7.0
! pip install -U transformers

     |████████████████████████████████| 3.8MB 3.3MB/s 
     |████████████████████████████████| 573kB 3.4MB/s 
     |████████████████████████████████| 3.7MB 47.4MB/s 
     |████████████████████████████████| 890kB 49.0MB/s 
     |████████████████████████████████| 1.0MB 43.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=33ab07f0d3097db2af01575ed326ad36ca9c6aa4172db2e801b7f0e212eaf2c9
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses
  Found existing installation: tokenizers 0.7.0
    Uninstalling tokenizers-0.7.0:
      Successfully uninstalled tokenizers-0.7.0


In [0]:
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
tqdm.pandas()
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors

In [0]:
DATA_FOLDER = "/content/drive/My Drive/course/DATA 2040/DATA2040_FinalProject/data" if IS_COLAB else "../../data"

In [0]:
import sys
HELPER_PATH = "/content/drive/My Drive/course/DATA 2040/DATA2040_FinalProject/utils" if IS_COLAB else "../utils"
sys.path.append(HELPER_PATH)
from helpers import regular_encode, roc_auc, RocAucEvaluation

In [7]:
os.listdir(DATA_FOLDER)

['jigsaw-toxic-comment-train-processed-seqlen128.csv',
 'jigsaw-toxic-comment-train.csv',
 'jigsaw-unintended-bias-train-processed-seqlen128.csv',
 'jigsaw-unintended-bias-train.csv',
 'sample_submission.csv',
 'test-processed-seqlen128.csv',
 'test.csv',
 'validation-processed-seqlen128.csv',
 'validation.csv',
 'submission.csv']

## TPU configuration

In [8]:
# Detect hardware, return appropriate distribution strategy
# Change the runtime type to TPU if you are on colab or Kaggle
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Running on TPU  grpc://10.63.245.154:8470
INFO:tensorflow:Initializing the TPU system: grpc://10.63.245.154:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.63.245.154:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


## Load data

In [0]:
train = pd.read_csv(DATA_FOLDER + "/jigsaw-toxic-comment-train.csv")
valid = pd.read_csv(DATA_FOLDER + '/validation.csv')
test = pd.read_csv(DATA_FOLDER + '/test.csv')
sub = pd.read_csv(DATA_FOLDER + '/sample_submission.csv')

In [0]:
# We drop the subtypes of toxic comments because we do not care about them
train.drop(['severe_toxic','obscene','threat','insult','identity_hate'],axis=1,inplace=True)

In [0]:
## Use a small data size for demonstration
train = train.loc[:12000,:] 

## Data Preprocessing

### Tokenize the dataset

In [12]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

In [0]:
# MODEL = 'bert-base-uncased'
# # MODEL = 'jplu/tf-xlm-roberta-base'
# # First load the real tokenizer
# tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [0]:
AUTO = tf.data.experimental.AUTOTUNE

# Configuration
EPOCHS = 2
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 192

In [16]:
%%time
x_train = regular_encode(train.comment_text.values, tokenizer, maxlen=MAX_LEN)
x_valid = regular_encode(valid.comment_text.values, tokenizer, maxlen=MAX_LEN)
x_test = regular_encode(test.content.values, tokenizer, maxlen=MAX_LEN)

y_train = train.toxic.values
y_valid = valid.toxic.values

CPU times: user 1min 45s, sys: 267 ms, total: 1min 45s
Wall time: 1min 45s


### Build the `Dataset` objects

In [0]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

## Baseline Model

In [0]:
def build_model(transformer, max_len=512):
    """
    https://www.kaggle.com/xhlulu/jigsaw-tpu-distilbert-with-huggingface-and-keras
    """
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    embed = transformer.weights[0].numpy()
    embedding = Embedding(np.shape(embed)[0], np.shape(embed)[1],
                          input_length=max_len, weights=[embed],
                          trainable=False)(input_word_ids)
#     sequence_output = transformer(input_word_ids)[0]
#     cls_token = sequence_output[:, 0, :]
    out = K.sum(embedding, axis=2)
    out = Dense(128, activation='relu')(out)
    out = Dense(1, activation='sigmoid')(out)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

### Define callbacks

In [0]:
def callback():
    cb = []

    reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss',  
                                    factor=0.3, patience=3, 
                                    verbose=1, mode='auto', 
                                    epsilon=0.0001, cooldown=1, min_lr=0.000001)
    cb.append(reduceLROnPlat)
    log = CSVLogger('log.csv')
    cb.append(log)

    RocAuc = RocAucEvaluation(validation_data=(x_valid, y_valid), interval=1)
    cb.append(RocAuc)
    
    return cb

In [0]:
%%time
with strategy.scope():
#     transformer_layer = TFAutoModel.from_pretrained(MODEL)
    transformer_layer = transformers.TFDistilBertModel.\
    from_pretrained('distilbert-base-multilingual-cased')

In [22]:
with strategy.scope():
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 192)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 192, 768)          91812096  
_________________________________________________________________
tf_op_layer_Sum (TensorFlowO [(None, 192)]             0         
_________________________________________________________________
dense (Dense)                (None, 128)               24704     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 91,836,929
Trainable params: 24,833
Non-trainable params: 91,812,096
_________________________________________________________________


### Two-stage training

In [27]:
n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    # callbacks=callback(),
    epochs=EPOCHS
)

Epoch 1/2
93/93 [==============================] - 3s 33ms/step - accuracy: 0.8490 - loss: 0.5725 - val_accuracy: 0.7976 - val_loss: 0.8751
Epoch 2/2
93/93 [==============================] - 3s 32ms/step - accuracy: 0.8553 - loss: 0.5410 - val_accuracy: 0.8019 - val_loss: 0.8580


In [29]:
n_steps = x_valid.shape[0] // BATCH_SIZE
train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    # callbacks=callback(),
    epochs=EPOCHS
)

Epoch 1/2
62/62 [==============================] - 1s 12ms/step - accuracy: 0.7675 - loss: 0.8039
Epoch 2/2
62/62 [==============================] - 1s 12ms/step - accuracy: 0.7713 - loss: 0.7795


## Inference and Submission

In [30]:
sub['toxic'] = model.predict(test_dataset, verbose=1)

499/499 [==============================] - 5s 10ms/step


In [0]:
sub.to_csv(DATA_FOLDER + '/submission.csv', index=False)

In [32]:
sub.head()

,id,toxic
0,0,0.048801
1,1,0.004734
2,2,0.006147
3,3,0.032414
4,4,0.003842
